### Приложение 5

#### Логическое резервное копирование

Логическое резервное копирование можно выполнить утилитой `pg_dump` для копирования конкретной базы данных на сервере или утилитой команды `pg_dumpall` для копирования всего кластера целиком.

In [1]:
# Видим что у нас всего одна база данных postgres 
# за исключением двух шаблонов template0 template1

!psql -U postgres -c "\l"

                                                 List of databases
   Name    |  Owner   | Encoding |   Collate   |    Ctype    | ICU Locale | Locale Provider |   Access privileges   
-----------+----------+----------+-------------+-------------+------------+-----------------+-----------------------
 postgres  | postgres | UTF8     | en_SG.UTF-8 | en_SG.UTF-8 |            | libc            | 
 template0 | postgres | UTF8     | en_SG.UTF-8 | en_SG.UTF-8 |            | libc            | =c/postgres          +
           |          |          |             |             |            |                 | postgres=CTc/postgres
 template1 | postgres | UTF8     | en_SG.UTF-8 | en_SG.UTF-8 |            | libc            | =c/postgres          +
           |          |          |             |             |            |                 | postgres=CTc/postgres
(3 rows)



In [2]:
# Создам таблицу test и наполню её данными!

!psql -U postgres -d template1 -c "DROP DATABASE postgres;"
!psql -U postgres -d template1 -c "CREATE DATABASE postgres;"
!psql -U postgres -c "CREATE TABLE test(id integer);"
!psql -U postgres -c "INSERT INTO test VALUES (1), (2), (3)"
!psql -U postgres -c "SELECT * FROM test;"

DROP DATABASE
CREATE DATABASE
CREATE TABLE
INSERT 0 3
 id 
----
  1
  2
  3
(3 rows)



In [3]:
# Посмотрим справку по резервному копированию pg_dump

!pg_dump --help

pg_dump dumps a database as a text file or to other formats.

Usage:
  pg_dump [OPTION]... [DBNAME]

General options:
  -f, --file=FILENAME          output file or directory name
  -F, --format=c|d|t|p         output file format (custom, directory, tar,
                               plain text (default))
  -j, --jobs=NUM               use this many parallel jobs to dump
  -v, --verbose                verbose mode
  -V, --version                output version information, then exit
  -Z, --compress=0-9           compression level for compressed formats
  --lock-wait-timeout=TIMEOUT  fail after waiting TIMEOUT for a table lock
  --no-sync                    do not wait for changes to be written safely to disk
  -?, --help                   show this help, then exit

Options controlling the output content:
  -a, --data-only              dump only the data, not the schema
  -b, --blobs                  include large objects in dump
  -B, --no-blobs               exclude large objects in d

In [4]:
# Выполним резервное копирование и 
# сохраним в файлик dump.txt

!pg_dump -U postgres -d postgres -f dump.txt
!ls -l dump.txt

-rw-r--r-- 1 evgeny evgeny 892 Jun  8 22:07 dump.txt


In [5]:
# прочитаем пару первых строк файла

!cat dump.txt

--
-- PostgreSQL database dump
--

-- Dumped from database version 15.6 (Debian 15.6-0+deb12u1)
-- Dumped by pg_dump version 15.6 (Debian 15.6-0+deb12u1)

SET statement_timeout = 0;
SET lock_timeout = 0;
SET idle_in_transaction_session_timeout = 0;
SET client_encoding = 'UTF8';
SET standard_conforming_strings = on;
SELECT pg_catalog.set_config('search_path', '', false);
SET check_function_bodies = false;
SET xmloption = content;
SET client_min_messages = warning;
SET row_security = off;

SET default_tablespace = '';

SET default_table_access_method = heap;

--
-- Name: test; Type: TABLE; Schema: public; Owner: postgres
--

CREATE TABLE public.test (
    id integer
);


ALTER TABLE public.test OWNER TO postgres;

--
-- Data for Name: test; Type: TABLE DATA; Schema: public; Owner: postgres
--

COPY public.test (id) FROM stdin;
1
2
3
\.


--
-- PostgreSQL database dump complete
--



Резервное копирование выполнено. Попрактикуюсь в восстановлении базы данных из резервной копии.

In [6]:
# Сначала дропну целевую базу данных и создам чистую

!psql -U postgres -d template1 -c "DROP DATABASE postgres;"
!psql -U postgres -d template1 -c "CREATE DATABASE postgres;"

DROP DATABASE
CREATE DATABASE


In [7]:
# А теперь восстановлю из дампа

!cat dump.txt | psql -U postgres -d postgres

SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
CREATE TABLE
ALTER TABLE
COPY 3


In [8]:
# Убедимся что данные восстановились

!psql -U postgres -c "SELECT * FROM test;"

 id 
----
  1
  2
  3
(3 rows)



###  Репликация

In [10]:
# Настройки по умолчанию позволяют 
# использовать протокол репликации:

# Посмотрим настройки на нашем сервере СУБД

!psql -U postgres -c "SHOW wal_level;"
!psql -U postgres -c "SHOW max_wal_senders;"

 wal_level 
-----------
 replica
(1 row)

 max_wal_senders 
-----------------
 10
(1 row)



In [11]:
# Посмотрим позволяют ли настройки подключение 
# по протоколу репикации

# Выясним где лежит файл pg_hba.conf

!psql -U postgres -c "SHOW hba_file;"

              hba_file               
-------------------------------------
 /etc/postgresql/15/main/pg_hba.conf
(1 row)



In [15]:
# Посмотрим его параметры 

!echo 314159 | sudo -S tail /etc/postgresql/15/main/pg_hba.conf

В самом конце файла видим параметр для локального подключения
* `trust` 

Значит всё в порядке

Создадим автономную резервную копию. С ключом `-R` утилита сформирует необходимые для репликации конфигурационные параметры.

In [18]:
!pg_basebackup -U postgres --pgdata=/home/evgeny/Projects/internship/basebackup -R

In [20]:
# Посмотрим что там

!ls -l /home/evgeny/Projects/internship/basebackup

total 216
-rw------- 1 evgeny evgeny    225 Jun  8 22:22 backup_label
-rw------- 1 evgeny evgeny 138237 Jun  8 22:22 backup_manifest
drwx------ 5 evgeny evgeny   4096 Jun  8 22:22 base
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 global
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_commit_ts
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_dynshmem
drwx------ 4 evgeny evgeny   4096 Jun  8 22:22 pg_logical
drwx------ 4 evgeny evgeny   4096 Jun  8 22:22 pg_multixact
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_notify
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_replslot
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_serial
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_snapshots
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_stat
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_stat_tmp
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_subtrans
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_tblspc
drwx------ 2 evgeny evgeny   4096 Jun  8 22:22 pg_twophase
-r

In [24]:
!echo 314159 | sudo -S mv /home/evgeny/Projects/internship/basebackup/ /var/lib/postgresql/15/replica

[sudo] password for evgeny: 

In [33]:
# Меня владельца файлов на postgres

!echo 314159 | sudo -S chown postgres:postgres -R /var/lib/postgresql/15/replica

[sudo] password for evgeny: 

In [35]:
# Проверяю правльность действий

!echo 314159 | sudo -S ls -l /var/lib/postgresql/15/replica

In [40]:
# Файл конфигурации реплики

!echo 314159 | sudo -S cat /var/lib/postgresql/15/replica/postgresql.auto.conf

Реплика сервера будет доступна сразу после выполнения команды 

`pg_ctlcluster 15 replica start`

Все изменения на мастер-сервере будут транслироваться на реплику